In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (ConfusionMatrixDisplay, confusion_matrix,
                             f1_score, precision_score, recall_score)
from sklearn.model_selection import (LeaveOneOut, LeavePOut, StratifiedKFold,
                                     cross_val_predict)
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

In [ ]:
from resolve_path import ajuste_path

In [ ]:
path = "data/util/"
path = ajuste_path(path)

df = pd.read_csv(path + "dataset_treinamento.csv")
df.info()

## Preparando o dataframe para o modelo

### Encoding do local

In [ ]:
label_encoder = LabelEncoder()

# locais antes do encoding
print(df["Local de instalação"].nunique())

df["local encoded"] = label_encoder.fit_transform(df["Local de instalação"])
df = df.drop(columns=["Local de instalação"])

# locais encoded
print(df["local encoded"].nunique())

### Escolhendo as colunas utilizadas

In [ ]:
print(df["binario acidentes"].value_counts())
print(df["Quantidade de Acidentes"].sum())

columns = ["Ano", "Mes", "HH total", "local encoded", "binario acidentes"]
df = df[columns]

## Matriz de correlação entre as colunas escolhidas

In [ ]:
corr = df.drop(columns="Ano").corr()

# list(plt.colormaps)
sns.heatmap(corr, cmap='RdBu', annot=True, vmin=-1, vmax=1)
plt.title("Matriz de Correlação")
plt.xticks(rotation=45)

plt.show()

## Definindo os datasets

### Escolhendo as features e o target

In [ ]:
X_columns = ["Ano", "Mes", "HH total",
             "local encoded", "Temp_Mean", "Precipitation_Sum"]
y_column = "binario acidentes"

X = df[X_columns]
y = df[y_column]

## Modelo

### Modelos

In [ ]:
models = {
    "Logistic Regression": {
        "model": LogisticRegression(),
        "probs": [],
        "preds": [],
    },
    "Linear Discriminant Analysis": {
        "model": LinearDiscriminantAnalysis(),
        "probs": [],
        "preds": [],
    },
    "SVC": {
        "model": SVC(probability=True, class_weight="balanced"),
        "probs": [],
        "preds": [],
    },
    "Naive Bayes": {
        "model": GaussianNB(),
        "probs": [],
        "preds": [],
    },
    "GBM": {
        "model":  GradientBoostingClassifier(),
        "probs": [],
        "preds": [],
    },
    "Random Forest": {
        "model": RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced"),
        "probs": [],
        "preds": [],
    },
}

### Cross validation

In [ ]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
loo = LeaveOneOut()
lpo = LeavePOut(p=5)

### Treino e Predição

In [ ]:
X = X.drop(columns=["Ano"], axis=1)
for model_name, m in models.items():
    model = m["model"]
    threshold = 0.5

    print(f"Model: {model_name}")
    m["probs"] = cross_val_predict(
        model, X, y, cv=skfold, method="predict_proba")[:, 1]
    m["preds"] = (m["probs"] >= threshold).astype(int)

### Distribuição normal das probabilidades

In [ ]:
for model_name, m in models.items():
    print(f"Model: {model_name}")

    max = m["probs"].max()
    freq_probs = pd.Series(m["probs"]).value_counts(
        bins=np.arange(0, max + 0.01, 0.01)).sort_index()
    print(freq_probs[freq_probs > 0])
    print("\n")

### Avaliação das métricas básicas

In [ ]:
for model_name, m in models.items():
    print(f"Model: {model_name}")
    print(f"Precision: {precision_score(y, m['preds'])}")
    print(f"Recall: {recall_score(y, m['preds'])}")
    print(f"F1 Score: {f1_score(y, m['preds'])}")

    cm = confusion_matrix(y_true=y, y_pred=m["preds"])

    cm_disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=["Negativo", "Positivo"])
    cm_disp.plot(cmap="Blues")

    plt.title(model_name)
    plt.show()